In [1]:
import wandb
import re

api = wandb.Api()

In [2]:
entity = "ianc-uc-berkeley"    # e.g., "myusername" or "myteam"
project = "vit-scratch-sim"  # e.g., "myproject"
job_name = "cube_mae_low_res_vit_v2"

In [3]:
env_names = [
    "cube",
    "peg",
    "slot",
    "hook",
    "pour",
    "thread",
]
policy_names = [
    "vit_v2",
    "low_res_vit_v2",
    "foveated_v2",
    "2stage_foveated_v3",
    "mae_vit_v2",
    "mae_low_res_vit_v2",
    "mae_foveated_v2",
    "mae_2stage_foveated_v3",
]

In [ ]:
from tqdm import tqdm
run_data = {}
runs = api.runs(f"{entity}/{project}")
for run in tqdm(runs):
    history = run.history(pandas=True)
    # Get columns that match patterns
    distractor_cols = [col for col in history.columns if re.match(r'eval/pc_success.*distractors', col)][0]
    iid_cols = [col for col in history.columns if re.match(r'eval/pc_success.*iid', col)][0]
    data = {
        "standard": history[iid_cols].max(),
        "distractors": history[distractor_cols].max(),
    }
    run_data[run.name] = data


100%|██████████| 79/79 [00:25<00:00,  3.07it/s]


In [12]:
for key in ["standard", "distractors"]:
    print(key)
    for env_name in env_names:
        row = ""
        for policy_name in policy_names:
            row += str(int(run_data[f"{env_name}_{policy_name}"][key])) + " "
        row += "//"
        print(row)



standard
46 76 68 66 66 36 68 60 //
12 18 26 28 10 22 34 22 //
54 44 56 52 64 54 57 64 //
32 24 14 54 52 30 28 57 //
12 30 68 38 32 34 46 30 //
24 32 56 56 48 40 68 70 //
distractors
72 98 68 100 100 100 90 100 //
26 32 26 32 40 32 38 32 //
56 57 60 60 57 66 64 70 //
28 18 12 56 68 56 30 57 //
34 60 78 84 68 78 80 92 //
57 48 62 74 84 66 74 92 //


In [ ]:

# runs = api.runs(f"{entity}/{project}")
# print("All run names:")
# for r in runs:
#     print(r.name)

for run in runs:
    print(f"Run ID: {run.id}")
    print(f"Config: {run.config}")
    print(f"Summary metrics: {run.summary}")
    # To get full history of a metric
    history = run.history(keys=["loss", "accuracy"], pandas=True)  # keys optional
    print(history)
